GitHub repo for this: https://github.com/reuven/2024-09September-08-decorators

# Decorators

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

In [1]:
# let's assume that we have two simple functions, a and b, which return something very simple

def a():
    return f'A!\n'

def b():
    return f'B!\n'

In [2]:
print(a())
print(b())

A!

B!



In [3]:
# our company has mandated that we put dashed lines above and below everything we print
# one option is for us to rewrite the functions

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [5]:
# the problem is that we will need to repeat this for every function we write!

# how can I rewrite this, such that I repeat myself as little as possible?
# option 1: write a function that takes our function as input, and returns the lines + our function's output

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'
    
print(with_lines(a))  # I'm not calling a
print(with_lines(b))  # I'm not calling b, either!

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [6]:
# the problem with the above is that we broke the existing/older API
# can we accomplish our goals, while keeping the old API?

# option 2: write a function that returns a function
# this is known as a closure -- we call a function, it returns a function, and then we assign that returned function to a variable

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
with_lines_b = with_lines(b)  
    
print(with_lines_a())  
print(with_lines_b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# How many functions?

1. We have the original `a` and `b` functions
2. We have `with_lines`, which we call, passing `a` and `b` to it
3. We have two functions, `with_lines_a` and `with_lines_b`, that we got back from calling `with_lines` with `a` and `b`

This is fine, but we can do even better -- we can assign the returned function from `with_lines(a)` not to `with_lines_a`, but rather to `a`. Similarly, we can assign the result of `with_lines(b)` back to `b`. As soon as we do that, we don't have direct access to the original `a` and `b` functions.

In [7]:
# option 3: don't define with_lines_a and with_lines_b. Rather, just assign back to a and b
# at this point, we have returned the original API, where we can call a() and b(), to working order

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [8]:
# Finally, we can use Python's decorator syntax
# this is identical in functionality to what we just did and saw, but it looks a bit different


lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines    # this line is equivalent to line 15
def a():
    return f'A!\n'
# a = with_lines(a)   

@with_lines   # this is equivalent to line 20
def b():
    return f'B!\n'
# b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# Implications of this syntax

1. Don't have both line 12 + 15! There's normally no reason for line 15, but if you're teaching decorators, forgetting to comment out one of them can be not good.
2. If you want to apply the same decorator to a number of different functions, you must say @decorator above each definition.

# Can we decorate anything?

Yes, we can now decorate any function so long as it takes *zero* arguments.




In [14]:
lines = '-' * 60 + '\n'

def with_lines(func):    # this is called at function-definition time
    def wrapper(*args):  # this is called once for each time the decorated function is invoked
        print(locals())
        return f'{lines}{func(*args)}{lines}'  # here, we unroll *args into zero or more arguments
    return wrapper

@with_lines    # this line is equivalent to line 15
def a():
    return f'A!\n'

@with_lines   # this is equivalent to line 20
def b():
    return f'B!\n'

@with_lines
def addition(first, second):
    return f'{first} + {second} = {first+second}\n'

print(a())  
print(b()) 
print(addition(3, 5))

{'args': (), 'func': <function a at 0x11172a480>}
------------------------------------------------------------
A!
------------------------------------------------------------

{'args': (), 'func': <function b at 0x11172a3e0>}
------------------------------------------------------------
B!
------------------------------------------------------------

{'args': (3, 5), 'func': <function addition at 0x111728d60>}
------------------------------------------------------------
3 + 5 = 8
------------------------------------------------------------



# What happens to `addition`?

1. We define the `addition` function in lines 17-18, as per usual.
2. After its definition is complete, then Python runs `addition = with_lines(addition)`. It does this thanks to the `@with_lines` on line 16, which decorates the function on lines 17-18.
3. That runs `with_lines`, passing it `addition` as an argument.
4. The only thing that `with_lines` does is define `wrapper` and then return it. Note that `wrapper` is not invoked at this stage!
5. Whatever `with_lines` returns (`wrapper` in this case) is assigned back to `addition`, which we defined back in step 1, on lines 17-18.
6. When we invoke `addition`, we're really invoking `wrapper`, which grabs `func` from the local variables of its enclosing function, invokes the function, and then returns its value inside of a dashed string.

# Who needs this? Where do we use decorators?

1. If I have several functions that need to do the same thing, I can extract that functionality into a decorator, and simplify my code.
2. Timing of functions
3. Logging of functions
4. Security wrappers
5. Setting values before/after a function runs
6. Check inputs and/or filter them
7. Check outputs and/or filter them

Decorators allow us to hijack a function both when it is defined (then, the outer function is run) and when it is invoked (then, the inner function is run). These are fully fledged functions that we can use however we want.

Some examples in the Python world of where we use decorators:
- `@property`
- Pytest, many of its things use decorators
- Flask or Django
- FastAPI
- SQLAlchemy

# How do we write a decorator?

We're going to need an outer function and an inner function.

1. The outer function, the decorator, takes one argument, `func`. The argument is the function that will be decorated. The outer function is invoked *once*, just after the decorated function is defined.
2. The inner function, typically called `wrapper`, which takes `*args` (or `**kwargs`). This function is invoked once for each time the original (decorated) function is invoked. It can do whatever it wants with inputs and outputs.
3. After defining the inner function, the outer function returns `wrapper`.
4. Once that's done, we can decorate any function we want.

# Python tutor visualization of this decorator

https://pythontutor.com/render.html#code=lines%20%3D%20'-'%20*%2060%20%2B%20'%5Cn'%0A%0Adef%20with_lines%28func%29%3A%20%20%0A%20%20%20%20def%20wrapper%28*args%29%3A%20%20%20%20%23%20this%20function%20takes%20any%20number%20of%20positional%20arguments%0A%20%20%20%20%20%20%20%20return%20f'%7Blines%7D%7Bfunc%28*args%29%7D%7Blines%7D'%20%20%23%20here,%20we%20unroll%20*args%20into%20zero%20or%20more%20arguments%0A%20%20%20%20return%20wrapper%0A%0A%40with_lines%20%20%20%20%23%20this%20line%20is%20equivalent%20to%20line%2015%0Adef%20a%28%29%3A%0A%20%20%20%20return%20f'A!%5Cn'%0A%0A%40with_lines%20%20%20%23%20this%20is%20equivalent%20to%20line%2020%0Adef%20b%28%29%3A%0A%20%20%20%20return%20f'B!%5Cn'%0A%0A%40with_lines%0Adef%20addition%28first,%20second%29%3A%0A%20%20%20%20return%20f'%7Bfirst%7D%20%2B%20%7Bsecond%7D%20%3D%20%7Bfirst%2Bsecond%7D%5Cn'%0A%0Aprint%28a%28%29%29%20%20%0Aprint%28b%28%29%29%20%0Aprint%28addition%283,%205%29%29&cumulative=false&curInstr=47&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false

# Exercise: Shouter

1. Write a decorator, `shouter`, that decorates functions that return strings.
2. Any such function's output will return in `ALL CAPS` and with an explanation point at the end.
Example:

```python
@shouter
def hello(name):
    return f'Hello, {name}'

print(hello('Reuven'))   # output HELLO, REUVEN!   